
## Mod 6 - Week 8 - Coding Homework

In [3]:
!pip install statsmodels

import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.spatial.distance import mahalanobis 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 43.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [statsmodels] [statsmodels]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


## Q1 

In [8]:

import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load dataset
df = pd.read_csv('homework_8.1.csv')

# Step 1: Estimate propensity scores using logistic regression (Z → X)
X = sm.add_constant(df['Z'])
y = df['X']
logit_model = sm.Logit(y, X).fit(disp=0)
df['p'] = logit_model.predict(X)

# Step 2: Calculate inverse probability weights
df['w'] = np.where(df['X'] == 1, 1 / df['p'], 1 / (1 - df['p']))

# Step 3: Compute weighted average treatment and control outcomes
treated = df[df['X'] == 1]
control = df[df['X'] == 0]

Y1 = np.average(treated['Y'], weights=treated['w'])
Y0 = np.average(control['Y'], weights=control['w'])

ATE_ipw = Y1 - Y0

print('IPW ATE (Q1):', round(float(ATE_ipw), 3)) 


IPW ATE (Q1): 2.268


## Q2

In [ ]:

first_three_p = df['p'].head(3).round(2).tolist()
print('First three propensity scores (Q2):', first_three_p)


First three propensity scores (Q2): [0.84, 0.59, 0.71]


## Q3 

In [11]:
# Load data
df2 = pd.read_csv('homework_8.2.csv')

treated2 = df2[df2['X'] == 1].copy()
control2 = df2[df2['X'] == 0].copy()

# Inverse covariance matrix for [Z1, Z2] from full sample
Z = df2[['Z1','Z2']].to_numpy()
inv_cov = np.linalg.inv(np.cov(Z.T))

def mahalanobis_vectorized(u, V_inv, X):
    diff = X - u
    left = diff @ V_inv
    return np.sqrt(np.sum(left * diff, axis=1))

treated_Z = treated2[['Z1','Z2']].to_numpy()
treated_Y = treated2['Y'].to_numpy()
control_Z = control2[['Z1','Z2']].to_numpy()
control_Y = control2['Y'].to_numpy()

# Nearest-neighbor matching (with replacement)
diffs = []
nearest_dists = []
for i, u in enumerate(treated_Z):
    dists = mahalanobis_vectorized(u, inv_cov, control_Z)
    j = np.argmin(dists)
    diffs.append(treated_Y[i] - control_Y[j])
    nearest_dists.append(dists[j])

ATE_mahal = float(np.mean(diffs))
print('Mahalanobis ATE (Q3):', round(ATE_mahal, 3))

Mahalanobis ATE (Q3): 3.438


## Q4 

In [14]:
idx_farthest = int(np.argmax(nearest_dists))
least_support_point = treated2.iloc[idx_farthest][['Z1','Z2']]
print('Least common support treated (Z1, Z2):', tuple(least_support_point.round(2))) 

Least common support treated (Z1, Z2): (2.7, 0.54)
